# Practical Exam: Insurance Upgrades

Travel Assured provides travel services to its customers. They are based in the United States.

Travel Assured provides everything from flights and hotel bookings to holiday insurance.

The sales team wants to sell upgrades to customers. So they can do this, it is vital that the data is clean, accurate and available for reporting.

They need your help to prepare some data before they start to run a new promotion.

**Database Schema**

The data you need is in the database named `insurance`.

![database_schema](schema.png)


# Task 1 

The sales team want to use customer information to target their new promotion. But they think the data may not be clean enough to use. 

The table below shows what the sales team expect the data types and format to be.

Write a query that makes the `customers` table match the description provided, including identifying and cleaning all invalid values. 

-  Your output should be a DataFrame with the name 'clean_data'. Do not modify the `customers` table.
-  Note that the DataLab environment formats dates as YYYY-MM-DD-hh-ss-SSS. 

| Column Name       | Description                                                      |
|-------------------|------------------------------------------------------------------|
| customer_id         | Unique integer (set by the database, can’t take any other value) |
| location       | State names as a lower case string                              |
| age        | Integer value giving age of customer                              |
| registration_date          | Date of first registration with company                    |

In [133]:
SELECT *
FROM customers
WHERE customer_id IS NULL 
	or location IS NULL 
	or age is NULL 
	or age is NULL 
	or registration_date IS NULL

Empty DataFrame
Columns: [customer_id, location, age, registration_date]
Index: []

We observe, we don't have any missing values

In [134]:
SELECT customer_id, COUNT(*) AS dup_count
FROM customers
GROUP BY customer_id
HAVING COUNT(*) > 1;


Empty DataFrame
Columns: [customer_id, dup_count]
Index: []

We observe, we don't have any duplicate rows

In [135]:
SELECT 
    column_name,
    data_type,
    character_maximum_length,
    is_nullable
FROM information_schema.columns
WHERE table_name = 'customers';


column_name data_type  character_maximum_length is_nullable
0        customer_id   integer                       NaN         YES
1           location      text                       NaN         YES
2                age      text                       NaN         YES
3  registration_date      date                       NaN         YES

We observe that the age column is currently stored as text (VARCHAR), but for proper analysis we want it as integers (INT)

In [136]:
SELECT age
FROM customers
limit 4

age
0  32 Years
1  23 Years
2  59 Years
3  39 Years

In [137]:
SELECT customer_id,LOWER(location) as location, CAST(SUBSTRING(age,1,2) as int) as age,registration_date
FROM customers


customer_id      location  age         registration_date
0               1      arkansas   32 2018-10-22 00:00:00+00:00
1               2         texas   23 2019-03-03 00:00:00+00:00
2               3      illinois   59 2018-04-12 00:00:00+00:00
3               4  pennsylvania   39 2018-09-05 00:00:00+00:00
4               5      new york   71 2019-01-29 00:00:00+00:00
...           ...           ...  ...                       ...
1495         1496    california   27 2020-04-06 00:00:00+00:00
1496         1497      kentucky   62 2021-01-06 00:00:00+00:00
1497         1498         texas   45 2019-08-14 00:00:00+00:00
1498         1499     wisconsin   25 2018-11-02 00:00:00+00:00
1499         1500      delaware   66 2020-08-01 00:00:00+00:00

[1500 rows x 4 columns]

# Task 2

The sales team wants to run a promotion on upgrades to international travel insurance policies. 

They only want to send this promotion to customers who have an active, US policy type.

Write a query to provide the `customer_id` and `start_date` for eligible customers.

In [138]:
WITH customers_for_promotion AS(
	SELECT policy_id,start_date
	FROM policy
	WHERE policy_type='US'
	AND active = 'TRUE'
)
SELECT s.customer_id, p.start_date
FROM sales as s
JOIN customers_for_promotion as p
ON s.policy_id = p.policy_id

customer_id                start_date
0              6 2022-10-09 00:00:00+00:00
1             11 2022-11-14 00:00:00+00:00
2             18 2022-12-29 00:00:00+00:00
3             24 2022-06-11 00:00:00+00:00
4             26 2022-09-02 00:00:00+00:00
..           ...                       ...
377         1482 2023-02-04 00:00:00+00:00
378         1489 2022-12-25 00:00:00+00:00
379         1490 2022-11-14 00:00:00+00:00
380         1493 2022-08-12 00:00:00+00:00
381         1496 2022-09-10 00:00:00+00:00

[382 rows x 2 columns]

# Task 3

After the promotion has been sent, the sales team will need to monitor the number of active policy holders by policy type who purchased an upgrade.

Write a query that returns the data for the sales team to monitor. Your output should include `policy_type` and `number_active` columns.

![database schema](schema.png)

In [139]:
select count(*)
from policy
where active='true'
group by policy_type

count
0    140
1    382

In [140]:
WITH active_customers AS (
    SELECT policy_id, start_date, policy_type
    FROM policy
    WHERE active = 'TRUE'
),
sales_policy AS (
    SELECT a.policy_id,
           a.policy_type,
           s.purchase_id,
           s.customer_id
    FROM active_customers AS a
    JOIN sales AS s
      ON s.policy_id = a.policy_id
)
SELECT s.policy_type,
       COUNT(DISTINCT s.customer_id) AS number_active
FROM sales_policy AS s
JOIN purchase AS p
  ON s.purchase_id = p.purchase_id
WHERE p.purchase_type = 'Upgrade'
GROUP BY s.policy_type;



policy_type  number_active
0  International             54
1             US            129